In [ ]:
# import regular expressins packge
# import numbers package
# supress system warnings
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["LOKY_MAX_CPU_COUNT"] = "1"
from sklearn.cluster import KMeans
from scipy.stats import mode
import matplotlib.pyplot as plt
import numpy as np
import re

In [ ]:
def readFile(fileName):
    file = open(fileName,'r',encoding="cp437")
    fileText = ""
    for line in file:
        fileText += line
    return fileText

In [ ]:
# preprocess text
def preProcess(text):
# Remove non-letter chars
    text = re.sub("[^a-zA-Z ]"," ", text)
# Change characters to lower
    text = text.lower()
    return text

In [ ]:
# generate a word set of a given text
def genReferenceWordList(texts,stopWords):
# concatenate the texts
    allText = ""
    for line in texts:
        allText += line
    # Generate a word list
    wordsList =  allText.split()
    # Generate a word set
    wordsSet =  set(wordsList)
    # Remove the stop words from the word list
    stopWordsList = stopWords.split()
    stopWordsSet = set(stopWordsList)
    refWordSet = wordsSet.difference(stopWordsSet)
    return list(refWordSet)

In [ ]:
# word frequency
def getWordFrequency(texts,refList):

    refListSize = len(refList)
    nTexts = len(texts)
    wordFreq = np.empty((nTexts,refListSize),dtype=np.int64)
    for i in range(nTexts):            # scan texts
        print("text" + str(i))
        for j in range(refListSize):      # scan words in dict
            wordFreq[i,j] = texts[i].count(refList[j])
    return wordFreq

In [ ]:
#Partition the text into a list of chunks of size windSize
def partitionText(text, windSize,label):
    nParts = len(text)//windSize
    chunks=[]
    labels = []
    for i in range(nParts):
        nextWind = i*windSize
        nextChunk = text[nextWind:nextWind+windSize]
        chunks += [nextChunk]
        labels += [label]
    return chunks,labels;

In [ ]:
# plot labels in bar chart
def plotLabels(labels,labelsType):

    plt.title("The " +  labelsType + " labels")
    plt.xlabel("partition#")
    plt.ylabel("label")
    plt.bar(range(len(labels)),labels)
    plt.show()

In [ ]:
# rearange predicted labels
# according to the most common true label
def rearangeLabels(trueLabels,predLabels,k):
    rearangedLabels = np.zeros(len(trueLabels))
    for i in range(k):
        predIs = np.where((i+1)==predLabels)[0]
        # Get the true labels in the predicted locations
        trueLabelsInLoc = trueLabels[predIs]
        # find the common label
        commonLabel,count = mode(trueLabelsInLoc)
        # put the comon true label in the predicted location
        rearangedLabels[predIs] = commonLabel
    return rearangedLabels

In [ ]:
bookTexts = []
#read  and preprocess files
bookTexts += [readFile('DB.txt')]
bookTexts += [readFile('HP.txt')]

In [ ]:
texts = []
for text in bookTexts:
    texts += [preProcess(text)]

In [ ]:
# Read stop words file - words that can be removed
stopWords = readFile('stopwords_en.txt')

In [ ]:
# Generate dictionary
refList = genReferenceWordList(texts,stopWords)

In [ ]:
#wind - chunks size
window = 50000

In [ ]:
chunks = []
trueLabels = []
for i, text in enumerate(texts):
    newChunks,newLabels = partitionText(text , window,i+1)
    chunks += newChunks
    trueLabels+=newLabels

In [ ]:
trueLabels = np.array(trueLabels)
plotLabels(trueLabels,"true")

In [ ]:
wordFreq = getWordFrequency(chunks,refList)

In [ ]:
# cluster with kmeans
k = len(bookTexts) # k: number of clusters

In [ ]:
# define the k-means object
kmeans = KMeans(n_clusters = k)
# compute the k-means model of the chunks
kmeans.fit(wordFreq)

In [ ]:
# predict the labels of the chunks
predLabels = kmeans.predict(wordFreq)
predLabels +=1

In [ ]:
# plot the k-means prediction
plotLabels(predLabels,"k-means")

In [ ]:
# rearange predicted labels according to the true labels
predictedLabels = rearangeLabels(trueLabels,predLabels,k)
plotLabels(predictedLabels,"k-means rearanged")